## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [7]:
from arcgis.gis import GIS
gis = GIS("home")



/opt/conda/lib/python3.11/site-packages/arcgis/gis/__init__.py:731: UserWarning: You are logged on as Ismael with an administrator role, proceed with caution.
  warnings.warn(


In [8]:
#Provide the item ID for a point feature layer with the following fields: level_ (integer), routes (integer), url (text), title (text). 
flayer = gis.content.get("b6c9487ef24d49d59912d8a569dbd1a1").layers[0]

In [9]:
#Provide the Mountain Project pages to process
urls = ['https://www.mountainproject.com/area/112389571/indiana']
urls = ['https://www.mountainproject.com/area/105708962/arizona',
        'https://www.mountainproject.com/area/105708964/new-mexico',
        'https://www.mountainproject.com/area/105873282/north-carolina',
        'https://www.mountainproject.com/area/105708956/colorado',
        'https://www.mountainproject.com/area/105708961/nevada',
        'https://www.mountainproject.com/area/105708957/utah',
        'https://www.mountainproject.com/area/105708966/washington',
        'https://www.mountainproject.com/area/105708959/california',
        'https://www.mountainproject.com/area/105907743/international']
urls = ['https://www.mountainproject.com/area/105905173/alabama',
       'https://www.mountainproject.com/area/105909311/alaska',
       'https://www.mountainproject.com/area/105901027/arkansas',
       'https://www.mountainproject.com/area/105806977/connecticut',
       'https://www.mountainproject.com/area/106861605/delaware',
       'https://www.mountainproject.com/area/111721391/florida',
       'https://www.mountainproject.com/area/105897947/georgia',
       'https://www.mountainproject.com/area/106316122/hawaii']
urls = ['https://www.mountainproject.com/area/106660030/europe',
        'https://www.mountainproject.com/area/106661603/south-america',
        'https://www.mountainproject.com/area/107833406/oceania',
        'https://www.mountainproject.com/area/106661469/north-america']
urls = ['https://www.mountainproject.com/area/105708960/wyoming',
       'https://www.mountainproject.com/area/105708968/wisconsin',
       'https://www.mountainproject.com/area/105855459/west-virginia',
       'https://www.mountainproject.com/area/105852400/virginia',
       'https://www.mountainproject.com/area/105891603/vermont',
       'https://www.mountainproject.com/area/105835804/texas',
       'https://www.mountainproject.com/area/105887760/tennessee',
       'https://www.mountainproject.com/area/105708963/south-dakota',
       'https://www.mountainproject.com/area/107638915/south-carolina',
       'https://www.mountainproject.com/area/106842810/rhode-island',
       'https://www.mountainproject.com/area/105913279/pennsylvania',
       'https://www.mountainproject.com/area/105854466/oklahoma',
       'https://www.mountainproject.com/area/105994953/ohio',
       'https://www.mountainproject.com/area/106598130/north-dakota',
       'https://www.mountainproject.com/area/105800424/new-york',
       'https://www.mountainproject.com/area/106374428/new-jersey',
       'https://www.mountainproject.com/area/105872225/new-hampshire',
       'https://www.mountainproject.com/area/116096758/nebraska',
       'https://www.mountainproject.com/area/105907492/montana',
       'https://www.mountainproject.com/area/105899020/missouri',
       'https://www.mountainproject.com/area/108307056/mississippi',
       'https://www.mountainproject.com/area/105812481/minnesota',
       'https://www.mountainproject.com/area/106113246/michigan',
       'https://www.mountainproject.com/area/105908062/massachusetts',
       'https://www.mountainproject.com/area/106029417/maryland',
       'https://www.mountainproject.com/area/105948977/maine',
       'https://www.mountainproject.com/area/116720343/louisiana',
       'https://www.mountainproject.com/area/105868674/kentucky',
       'https://www.mountainproject.com/area/107235316/kansas',
       'https://www.mountainproject.com/area/106092653/iowa',
       'https://www.mountainproject.com/area/105911816/illinois',
       'https://www.mountainproject.com/area/105708958/idaho']

In [10]:
import requests

def processURL(url, level,flayer, debug):

    if debug and level==0:
        display (f"Processing {url}")
        
    #Get HTML
    response = requests.get(url).text

    #Get Title
    title = url.replace('-',' ')
    title = title[title.rfind('/') + 1:len(title)].title()
            
    #Get coordinates (If no coordinates are found, exit)
    lat = 0
    lon = 0
    try:
        start = response.find('maps.google.com/maps?q=') + 23 
        if start == -1:
            display (f"    Info: No coordinates in {url}")
            return
        else:
            coordinates = response[start:start+response[start:start+50].find('&')]
            coords = coordinates.split(',')
            lat = coords[0]
            lon = coords[1]
    except:
        display (f"    Error while getting coords for {url}")
                
    #Get route count
    routeCount = 0
    try:
        start = 0
        while 1==1:
            start = response.find('<td class="route-score">',start)
            if start == -1: 
                break
            routeCount = routeCount + 1
            start = response.find('</td>',start)
    except:
        display (f"    Error getting route count for {url}")  
    
    #Add row to feature layer
    if debug: display ("Level: " + str(level) + " Coordinates:" + coordinates + " Route Count: " + str(routeCount) + " URL: " + url)
    try:
        myRow = {"attributes": 
               {
               "level_": level,
               "routes": routeCount,
               "url": url,
               "title":title}, 
               "geometry": 
               {"x": lon, "y": lat, "spatialReference":{"wkid": 4326}}}
        add_result = flayer.edit_features(adds = [myRow])
    except:
        display (f"    Error adding row to feature layer for {url}")
        
    #Process children recursively
    start = 0
    while 1==1:
        start = response.find('<div class="lef-nav-row">',start)
        if start == -1: 
            level = level - 1
            break
        start = response.find('<a href="',start)
        childURL = response[start + 9:start+response[start:start+500].find('">')]
        processURL(childURL, level + 1,flayer, debug)
    

In [ ]:
#Process Mountain Project URLs storing data in the feature layer
for url in urls:
    display (f"Processing {url}")
    processURL(url,0,flayer,False)
    
    display ('Done!')
